Database Schema Update

In [ ]:
CREATE TABLE Users (
    user_id INT PRIMARY KEY AUTO_INCREMENT,
    username VARCHAR(255) UNIQUE NOT NULL
);

CREATE TABLE Subjects (
    subject_id INT PRIMARY KEY AUTO_INCREMENT,
    user_id INT,
    subject_name VARCHAR(255) NOT NULL,
    CONSTRAINT unique_user_subject UNIQUE (user_id, subject_name),
    FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE
);

CREATE TABLE Chapters (
    chapter_id INT PRIMARY KEY AUTO_INCREMENT,
    subject_id INT,
    chapter_name VARCHAR(255) NOT NULL,
    CONSTRAINT unique_subject_chapter UNIQUE (subject_id, chapter_name),
    FOREIGN KEY (subject_id) REFERENCES Subjects(subject_id) ON DELETE CASCADE
);

CREATE TABLE Topics (
    topic_id INT PRIMARY KEY AUTO_INCREMENT,
    chapter_id INT,
    topic_name VARCHAR(255) NOT NULL,
    CONSTRAINT unique_chapter_topic UNIQUE (chapter_id, topic_name),
    FOREIGN KEY (chapter_id) REFERENCES Chapters(chapter_id) ON DELETE CASCADE
);

CREATE TABLE Contents (
    content_id INT PRIMARY KEY AUTO_INCREMENT,
    topic_id INT,
    content_text TEXT NOT NULL,
    FOREIGN KEY (topic_id) REFERENCES Topics(topic_id) ON DELETE CASCADE
);




insertion

In [ ]:
-- Adding users
INSERT INTO Users (username) VALUES ('john_doe');
INSERT INTO Users (username) VALUES ('jane_smith');

-- Adding subjects for users
INSERT INTO Subjects (user_id, subject_name) VALUES (1, 'Physics'); -- OK
INSERT INTO Subjects (user_id, subject_name) VALUES (1, 'Mathematics'); -- OK
INSERT INTO Subjects (user_id, subject_name) VALUES (2, 'Physics'); -- OK (same subject, different user)
INSERT INTO Subjects (user_id, subject_name) VALUES (1, 'Physics'); -- ERROR (duplicate subject for same user)

-- Adding chapters
INSERT INTO Chapters (subject_id, chapter_name) VALUES (1, 'Mechanics'); -- OK
INSERT INTO Chapters (subject_id, chapter_name) VALUES (1, 'Thermodynamics'); -- OK
INSERT INTO Chapters (subject_id, chapter_name) VALUES (1, 'Mechanics'); -- ERROR (duplicate chapter under same subject)

-- Adding topics
INSERT INTO Topics (chapter_id, topic_name) VALUES (1, 'Newton’s Laws'); -- OK
INSERT INTO Topics (chapter_id, topic_name) VALUES (1, 'Work and Energy'); -- OK
INSERT INTO Topics (chapter_id, topic_name) VALUES (1, 'Newton’s Laws'); -- ERROR (duplicate topic under same chapter)


Insert Content for a Topic

In [ ]:
INSERT INTO Contents (topic_id, content_text)
SELECT topic_id, 'This is content for Gravitation'
FROM Topics 
WHERE topic_name = 'Gravitation' 
AND chapter_id = (SELECT chapter_id FROM Chapters WHERE chapter_name = 'Mechanics');


Retrieve All Topics & Their Content for user_id = 1

In [ ]:
SELECT 
    S.subject_name, 
    C.chapter_name, 
    T.topic_name,
    COALESCE(CT.content_text, 'No Content Available') AS content
FROM Subjects S
JOIN Chapters C ON S.subject_id = C.subject_id
JOIN Topics T ON C.chapter_id = T.chapter_id
LEFT JOIN Contents CT ON T.topic_id = CT.topic_id  -- Include content
WHERE S.user_id = 1
ORDER BY S.subject_name, C.chapter_name, T.topic_name;


Insert Subject, Chapter, Topic & Content in One Query

In [ ]:
-- Step 1: Insert "Physics" if not exists
INSERT INTO Subjects (user_id, subject_name)
SELECT 1, 'Physics' 
WHERE NOT EXISTS (
    SELECT 1 FROM Subjects WHERE subject_name = 'Physics' AND user_id = 1
);

-- Step 2: Get subject_id
SET @subject_id = (SELECT subject_id FROM Subjects WHERE subject_name = 'Physics' AND user_id = 1);

-- Step 3: Insert "Mechanics" if not exists
INSERT INTO Chapters (subject_id, chapter_name)
SELECT @subject_id, 'Mechanics'
WHERE NOT EXISTS (
    SELECT 1 FROM Chapters WHERE chapter_name = 'Mechanics' AND subject_id = @subject_id
);

-- Step 4: Get chapter_id
SET @chapter_id = (SELECT chapter_id FROM Chapters WHERE chapter_name = 'Mechanics' AND subject_id = @subject_id);

-- Step 5: Insert "Gravitation" if not exists
INSERT INTO Topics (chapter_id, topic_name)
SELECT @chapter_id, 'Gravitation'
WHERE NOT EXISTS (
    SELECT 1 FROM Topics WHERE topic_name = 'Gravitation' AND chapter_id = @chapter_id
);

-- Step 6: Insert Content for "Gravitation"
SET @topic_id = (SELECT topic_id FROM Topics WHERE topic_name = 'Gravitation' AND chapter_id = @chapter_id);

INSERT INTO Contents (topic_id, content_text)
SELECT @topic_id, 'This is the content for Gravitation'
WHERE NOT EXISTS (
    SELECT 1 FROM Contents WHERE topic_id = @topic_id
);


Retrieve Subjects for user_id = 1

In [ ]:
SELECT subject_id, subject_name 
FROM Subjects 
WHERE user_id = 1;


Retrieve Chapters for subject_id = 1 (Belonging to user_id = 1)

In [ ]:
SELECT chapter_id, chapter_name 
FROM Chapters 
WHERE subject_id = 1;


Retrieve Topics for chapter_id = 1 (Under subject_id = 1, Belonging to user_id = 1)

In [ ]:
SELECT T.topic_id, T.topic_name 
FROM Topics T
JOIN Chapters C ON T.chapter_id = C.chapter_id
JOIN Subjects S ON C.subject_id = S.subject_id
WHERE C.chapter_id = 1 AND S.subject_id = 1 AND S.user_id = 1;


SQL Query to Get All Subjects, Chapters, and Topics for user_id = 1

In [ ]:
SELECT 
    S.subject_id, S.subject_name, 
    C.chapter_id, C.chapter_name, 
    T.topic_id, T.topic_name
FROM Subjects S
LEFT JOIN Chapters C ON S.subject_id = C.subject_id
LEFT JOIN Topics T ON C.chapter_id = T.chapter_id
WHERE S.user_id = 1
ORDER BY S.subject_id, C.chapter_id, T.topic_id;


In [ ]:
-- Example Output for user_id = 1
-- subject_id	subject_name	chapter_id	chapter_name	topic_id	topic_name
-- 1	Physics	1	Mechanics	1	Newton’s Laws
-- 1	Physics	1	Mechanics	2	Work & Energy
-- 1	Physics	2	Thermodynamics	NULL	NULL
-- 2	Mathematics	3	Algebra	3	Quadratic Equations
-- 2	Mathematics	3	Algebra	4	Matrices


Alternative: Grouped Result Using GROUP_CONCAT

In [ ]:
SELECT 
    S.subject_name,
    GROUP_CONCAT(DISTINCT C.chapter_name ORDER BY C.chapter_id SEPARATOR ', ') AS chapters,
    GROUP_CONCAT(DISTINCT T.topic_name ORDER BY T.topic_id SEPARATOR ', ') AS topics
FROM Subjects S
LEFT JOIN Chapters C ON S.subject_id = C.subject_id
LEFT JOIN Topics T ON C.chapter_id = T.chapter_id
WHERE S.user_id = 1
GROUP BY S.subject_id;


Find subject_id for "Physics" (for user_id = 1)

In [ ]:
SELECT subject_id 
FROM Subjects 
WHERE subject_name = 'Physics' AND user_id = 1;


Find chapter_id for "Mechanics" under "Physics"

In [ ]:
SELECT chapter_id 
FROM Chapters 
WHERE chapter_name = 'Mechanics' AND subject_id = X;


Insert "Gravitation" into the Topics Table

In [ ]:
INSERT INTO Topics (chapter_id, topic_name) 
VALUES (Y, 'Gravitation');


Full SQL Query (Without Manually Checking IDs)

In [ ]:
INSERT INTO Topics (chapter_id, topic_name)
VALUES (
    (SELECT chapter_id 
     FROM Chapters 
     WHERE chapter_name = 'Mechanics' 
     AND subject_id = (SELECT subject_id 
                       FROM Subjects 
                       WHERE subject_name = 'Physics' 
                       AND user_id = 1)
    ), 
    'Gravitation'
);


Full SQL Query (Handle Missing Entries)

In [ ]:
-- Step 1: Insert "Physics" if not exists
INSERT INTO Subjects (user_id, subject_name)
SELECT 1, 'Physics' 
WHERE NOT EXISTS (
    SELECT 1 FROM Subjects WHERE subject_name = 'Physics' AND user_id = 1
);

-- Step 2: Get subject_id for "Physics"
SET @subject_id = (SELECT subject_id FROM Subjects WHERE subject_name = 'Physics' AND user_id = 1);

-- Step 3: Insert "Mechanics" if not exists
INSERT INTO Chapters (subject_id, chapter_name)
SELECT @subject_id, 'Mechanics'
WHERE NOT EXISTS (
    SELECT 1 FROM Chapters WHERE chapter_name = 'Mechanics' AND subject_id = @subject_id
);

-- Step 4: Get chapter_id for "Mechanics"
SET @chapter_id = (SELECT chapter_id FROM Chapters WHERE chapter_name = 'Mechanics' AND subject_id = @subject_id);

-- Step 5: Insert "Gravitation" into Topics
INSERT INTO Topics (chapter_id, topic_name)
SELECT @chapter_id, 'Gravitation'
WHERE NOT EXISTS (
    SELECT 1 FROM Topics WHERE topic_name = 'Gravitation' AND chapter_id = @chapter_id
);


SQL Query to Get All Topics for user_id = 1

In [ ]:
SELECT 
    S.subject_name, 
    C.chapter_name, 
    T.topic_name
FROM Subjects S
JOIN Chapters C ON S.subject_id = C.subject_id
JOIN Topics T ON C.chapter_id = T.chapter_id
WHERE S.user_id = 1
ORDER BY S.subject_name, C.chapter_name, T.topic_name;
